In [68]:
from sgp4.api import jday
import datetime
import pandas as pd
from sgp4.api import Satrec
from sgp4 import omm
import numpy as np
%matplotlib notebook
from sklearn.cluster import DBSCAN
import plotly.graph_objects as go
from sklearn.neighbors import KernelDensity
from scipy.stats import multivariate_normal
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabasz_score


In [69]:
#Get date
current_time = datetime.datetime.now()
xd, xr = jday(current_time.year , current_time.month , current_time.day, current_time.hour, current_time.minute , current_time.second)
print(xd)
print(xr)

2460405.5
0.8096180555555555


In [70]:
with open('../Data/satellitedata.csv') as f:
    lines = omm.parse_csv(f)
    fields = [field for field in lines]
 

In [71]:
Satellite = []
result_satellite = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    if any([abs(coord) > 10_000 for coord in r]):
     continue
    Satellite.append(r)
    result_satellite.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])
    
    
  

data_satellite = np.array(Satellite)
data_satellite = data_satellite[~np.isnan(data_satellite).any(axis=1)]

result_satellite = pd.DataFrame(result_satellite,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()

fig = go.Figure(data=[go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2],
                                   mode='markers',name="Satellite", marker={"size":12, "color":"white"})])

fig.show()

In [72]:
normalization = np.linalg.norm(data_satellite, axis=1)[:, np.newaxis]
data_satellite_norm = data_satellite / normalization

fig = go.Figure(data=[go.Scatter3d(x=data_satellite_norm[:,0], y=data_satellite_norm[:,1], z=data_satellite_norm[:,2],
                                   mode='markers',name="Satellite", marker={"size":10, "color":"white"})])

fig.show()

In [73]:
# DBSCAN for cosmos garbage
with open('../Data/cosmos.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Cosmos_G = []
result_cosmos = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Cosmos_G.append(r)
    result_cosmos.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])

data_garbage_cosmos = np.array(Cosmos_G)
data_garbage_cosmos = data_garbage_cosmos[~np.isnan(data_garbage_cosmos).any(axis=1)]

result_cosmos = pd.DataFrame(result_cosmos,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers')])
fig.show()


model = DBSCAN(eps=800, min_samples=4)
model.fit_predict(data_garbage_cosmos)
pred_cosmos = model.fit_predict(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":pred_cosmos})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'gold'}))

fig.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)


number of cluster found: 38
cluster for each point:  [ 0 12 35 -1  1 10  2 -1  3  4  5  6  6  7 -1  4  3  8  4  9 -1  7  3  3
  3 -1  3 -1  3  3 33 10 -1 -1  4 -1 -1 36 -1 -1  3 10  9  3  9  0  3 -1
  7 -1 -1  8 -1  9  4 -1  3 -1  3 -1  6 -1 -1 19  6 11 12  4  3  4  3  4
 -1 -1 -1  0  4 -1  3  4  3  3 13  4 -1 31  3 14  3 14  6  3 -1 -1 15  4
 -1 -1 23 29 -1 -1  4 -1  4  4 14  3 15 12 27  4 -1 16 17 -1 -1 -1  9  3
  4  3 -1 36  4 28  4  3 18 19 -1 -1 -1  3  4 32 22 21 20 11 -1 12 19 -1
 -1 23 -1 -1  3  3 -1 -1  4 12 -1  0 -1  6  4  4  3 -1 34  3 -1 10 21 -1
 -1 -1 -1  4  8 27 -1 -1 22  4 25  3  4 -1  3 18 -1  4 -1 -1 -1 -1  3 -1
  5  3  4 -1 -1 -1 -1 -1 -1  3 -1 -1 20 -1 23 -1 24 -1 -1 -1 -1  4 21  3
  4  3  9  4 -1 -1 -1 25 -1 -1  1  3  9 35  4  4 -1  2 -1  4  3  4 26 -1
  4  3  3  3  3 -1 20  4  4 -1 19 29  3 30  3 12  7  0 25  4  3  3  6 -1
 -1  4  8  3 13 -1  3 -1 -1 26  4 15 12 -1  4 -1 10 27  4 -1  4 17 25  3
 -1 -1  4  6  3 -1  3 28  3  4 -1 -1 12 22  4 13 -1 -1 18 -1 29  3 -1  

In [74]:
# DBSCAN for iridium garbage
with open('../Data/iridium.csv') as f:
        lines = omm.parse_csv(f)
        fields = [field for field in lines]

Iridium_G = []
result_iridium = []

for field in fields:
    sat = Satrec()
    satellite = omm.initialize(sat, field)
    jd, fr = xd, xr
    e, r, v = sat.sgp4(jd, fr)
    Iridium_G.append(r)
    result_iridium.append([field['OBJECT_NAME'],field['OBJECT_ID'],field['EPOCH'], *r])
    
    

data_garbage_iridium = np.array(Iridium_G)
data_garbage_iridium = data_garbage_iridium[~np.isnan(data_garbage_iridium).any(axis=1)]


result_iridium = pd.DataFrame(result_iridium,columns=['OBJECT_NAME','OBJECT_ID','EPOCH','x','y','z']).dropna()


fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers')])
fig.show()

model = DBSCAN(eps=1570, min_samples=4)
model.fit_predict(data_garbage_iridium)
pred_iridium = model.fit_predict(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":12, "color":pred_iridium})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'gold'}))

fig.show()

print("number of cluster found: {}".format(len(set(model.labels_))))
print('cluster for each point: ', model.labels_)

number of cluster found: 8
cluster for each point:  [ 0  1 -1 -1  1 -1  1 -1  4  0 -1 -1  2  6 -1 -1  0  3 -1  6 -1 -1  1  0
 -1 -1 -1  2  1 -1  1  1  1 -1 -1  0 -1  0 -1  0  2  1  1 -1 -1  1  1  4
  1  3 -1 -1  1 -1 -1  4 -1  1  1  1 -1  0  0 -1 -1 -1  1 -1 -1  0  1  1
  1  6  2  1 -1  5 -1 -1  1  1  1  1 -1 -1  1 -1 -1 -1  5 -1  0 -1  0  1
  3  1  1  1 -1  6 -1  2 -1 -1  5  0  0  1  0  0  1  1 -1  1  0 -1 -1  1
 -1  1 -1  1 -1 -1  1  1 -1  0  3  3 -1  1  1 -1  1  1 -1  0  1  0  4 -1
 -1  2  0 -1  5  1  1 -1  1 -1 -1 -1  1  0  1 -1  1  1 -1  2  1  5 -1  0
 -1 -1]


In [75]:
#KDE for iridium garbage
model = KernelDensity(kernel='gaussian', bandwidth = 1500)
model.fit(data_garbage_iridium)
log_dens_iridium = model.score_samples(data_garbage_iridium)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":log_dens_iridium})])
fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'white'}))
fig.show()

In [76]:
#KDE for cosmos garbage
model = KernelDensity(kernel='gaussian', bandwidth = 1000)
model.fit(data_garbage_cosmos)
log_dens_cosmos = model.score_samples(data_garbage_cosmos)

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:,0], y=data_garbage_cosmos[:,1], z=data_garbage_cosmos[:,2],
                                   mode='markers',name="Garbage", marker={"size":8, "color":log_dens_cosmos})])

fig.add_trace(go.Scatter3d(x=data_satellite[:,0], y=data_satellite[:,1], z=data_satellite[:,2], mode='markers',marker_symbol='circle-open',name="Satellite", marker={"size":12, "color":'white'}))
fig.show()


In [77]:
# EM Algorithm for data_garbage_cosmos


# Filter out rows with inf or NaN values
data_garbage_cosmos = np.array([row for row in data_garbage_cosmos if np.isfinite(row).all()])

# Initialize parameters
n_components = 3
mu = np.random.rand(n_components, 3)  # Random initialization of means
sigma = [np.eye(3) for _ in range(n_components)]  # Initialization of covariance matrices as identity matrices
pi = np.ones(n_components) / n_components  # Initialization of mixture weights as a uniform distribution
z = np.zeros((len(data_garbage_cosmos), n_components))

def likelihood(x, mu, sigma):
    sigma = np.where(np.isinf(sigma) | np.isnan(sigma), 1e-6 * np.eye(3), sigma)
    return multivariate_normal(mean=mu, cov=sigma).pdf(x) + np.finfo(float).eps

# EM algorithm
for step in range(100):
    # E-step
    for k in range(n_components):
        z[:, k] = pi[k] * likelihood(data_garbage_cosmos, mu[k], sigma[k])
        z /= z.sum(axis=1, keepdims=True) + np.finfo(float).eps

    # M-step
    for k in range(n_components):
        weight = np.sum(z[:, k])
        mu[k] = np.sum(z[:, k, np.newaxis] * data_garbage_cosmos, axis=0) / weight
        diff = data_garbage_cosmos - mu[k]
        cov = np.dot(diff.T, z[:, k, np.newaxis] * diff) / weight
        cov += 1e-3 * np.eye(3)  # Add small noise to covariance matrix to avoid numerical instabilities
        sigma[k] = cov
        pi[k] = np.clip((weight + 1e-6) / (len(data_garbage_cosmos) + n_components * 1e-6), 0, 1)

# Visualization
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data')])
fig.add_trace(go.Scatter3d(x=mu[:, 0], y=mu[:, 1], z=mu[:, 2], mode='markers', marker=dict(size=8), name='Means'))
fig.show()

In [78]:
# EM Algorithm for data_garbage_iridium


# Filter out rows with inf or NaN values
data_garbage_iridium = np.array([row for row in data_garbage_iridium if np.isfinite(row).all()])

# Initialize parameters
n_components = 3
mu = np.random.rand(n_components, 3)  # Random initialization of means
sigma = [np.eye(3) for _ in range(n_components)]  # Initialization of covariance matrices as identity matrices
pi = np.ones(n_components) / n_components  # Initialization of mixture weights as a uniform distribution
z = np.zeros((len(data_garbage_iridium), n_components))

def likelihood(x, mu, sigma):
    sigma = np.where(np.isinf(sigma) | np.isnan(sigma), 1e-6 * np.eye(3), sigma)
    return multivariate_normal(mean=mu, cov=sigma).pdf(x) + np.finfo(float).eps

# EM algorithm
for step in range(100):
    # E-step
    for k in range(n_components):
        z[:, k] = pi[k] * likelihood(data_garbage_iridium, mu[k], sigma[k])
        z /= z.sum(axis=1, keepdims=True) + np.finfo(float).eps

    # M-step
    for k in range(n_components):
        weight = np.sum(z[:, k])
        mu[k] = np.sum(z[:, k, np.newaxis] * data_garbage_iridium, axis=0) / weight
        diff = data_garbage_iridium - mu[k]
        cov = np.dot(diff.T, z[:, k, np.newaxis] * diff) / weight
        cov += 1e-3 * np.eye(3)  # Add small noise to covariance matrix to avoid numerical instabilities
        sigma[k] = cov
        pi[k] = np.clip((weight + 1e-6) / (len(data_garbage_iridium) + n_components * 1e-6), 0, 1)

# Visualization
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:, 0], y=data_garbage_iridium[:, 1], z=data_garbage_iridium[:, 2], mode='markers', name='Data')])
fig.add_trace(go.Scatter3d(x=mu[:, 0], y=mu[:, 1], z=mu[:, 2], mode='markers', marker=dict(size=8), name='Means'))
fig.show()

In [79]:
#Calinski Harabasz for Cosmos
scores = []

for i in range(10, 21):
    kmeans = KMeans(n_clusters=i).fit(data_garbage_cosmos)
    
    score = calinski_harabasz_score(data_garbage_cosmos, kmeans.labels_)
    
    scores.append(score)
    
    
for i,score in enumerate(scores):
    print(f"n_clusters={i+10}, score={score}")

highest_score_index = scores.index(max(scores))

print(f"The highest score is {max(scores)} at i = {highest_score_index + 10}")

n_clusters=10, score=405.23336608036055
n_clusters=11, score=388.7737363498895
n_clusters=12, score=402.6691087929273
n_clusters=13, score=428.16526099071035
n_clusters=14, score=418.2149337634588
n_clusters=15, score=430.32272155030734
n_clusters=16, score=432.3639324102999
n_clusters=17, score=404.80935510593866
n_clusters=18, score=415.5225599156376
n_clusters=19, score=434.528198142698
n_clusters=20, score=413.5487715007604
The highest score is 434.528198142698 at i = 19


In [80]:
#K-Means for data_garbage_cosmos
kmeans = KMeans(n_clusters=20).fit(data_garbage_cosmos)


labels = kmeans.labels_

fig = go.Figure(data=[go.Scatter3d(x=data_garbage_cosmos[:, 0], y=data_garbage_cosmos[:, 1], z=data_garbage_cosmos[:, 2], mode='markers', name='Data', marker=dict(size=8, color=labels, colorscale='Viridis'))])

fig.add_trace(go.Scatter3d(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1], z=kmeans.cluster_centers_[:, 2],
                          mode='markers', name='Cluster Centers', marker=dict(size=12, color='red', opacity=0.7)))

fig.show()

In [81]:
#Calinski Harabasz for iridium
scores_for_iridium = []

for i in range(2, 11):
    kmeans = KMeans(n_clusters=i).fit(data_garbage_iridium)
    
    score = calinski_harabasz_score(data_garbage_iridium, kmeans.labels_)
    
    scores_for_iridium.append(score)
   

for i, score in enumerate(scores_for_iridium):
    print(f"n_clusters={i+2}, score={score}")

highest_score_index = scores_for_iridium.index(max(scores_for_iridium))

print(f"The highest score is {max(scores_for_iridium)} at i = {highest_score_index + 2}")

n_clusters=2, score=117.05051187610923
n_clusters=3, score=104.57668061631621
n_clusters=4, score=104.65371148425184
n_clusters=5, score=98.93231082645478
n_clusters=6, score=116.1564830556492
n_clusters=7, score=102.47569472800656
n_clusters=8, score=102.30775566005777
n_clusters=9, score=113.26091837269384
n_clusters=10, score=106.91507621082165
The highest score is 117.05051187610923 at i = 2


In [82]:
#K-means for data_garbage_iridium
kmeans = KMeans(n_clusters=6).fit(data_garbage_iridium)

labels = kmeans.labels_
fig = go.Figure(data=[go.Scatter3d(x=data_garbage_iridium[:,0], y=data_garbage_iridium[:,1], z=data_garbage_iridium[:,2],
                                   mode='markers',name="Garbage", marker={"size":12, "color":labels})])

fig.add_trace(go.Scatter3d(x=kmeans.cluster_centers_[:, 0], y=kmeans.cluster_centers_[:, 1], z=kmeans.cluster_centers_[:, 2],
                          mode='markers', name='Cluster Centers', marker=dict(size=12, color='red', opacity=0.7)))

fig.show()

In [83]:
result_iridium['Cluster'] = pred_iridium
result_iridium['Density'] = log_dens_iridium
result_iridium.to_csv('../Data/result_iridium.csv', index= False)
result_iridium

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z,Cluster,Density
0,IRIDIUM 33,1997-051C,2024-03-15T03:57:12.554496,47.404158,507.655180,7124.369606,0,-27.232020
1,IRIDIUM 33 DEB,1997-051L,2024-03-14T17:47:59.202816,-2006.538105,1077.858799,-6776.101070,1,-27.616915
2,IRIDIUM 33 DEB,1997-051N,2024-03-15T04:39:16.727904,4668.937876,-5325.227529,-889.835385,-1,-28.539352
3,IRIDIUM 33 DEB,1997-051P,2024-03-14T20:44:27.452832,4787.530220,-1505.118664,5070.949210,-1,-28.689858
4,IRIDIUM 33 DEB,1997-051Q,2024-03-14T15:11:01.815360,285.003291,-1320.496808,-6991.686559,1,-27.324273
...,...,...,...,...,...,...,...,...
165,IRIDIUM 33 DEB,1997-051ABX,2024-03-14T23:53:25.857024,-2117.976537,-6721.929817,902.811537,5,-28.671717
166,IRIDIUM 33 DEB,1997-051ACD,2024-03-14T07:15:03.155904,1693.936728,4742.579342,4955.360264,-1,-28.047597
167,IRIDIUM 33 DEB,1997-051ACE,2024-03-11T22:52:08.279904,900.509405,-558.113186,7030.727235,0,-27.240654
168,IRIDIUM 33 DEB,1997-051ACG,2024-03-14T13:33:50.161824,1891.217832,-130.301190,-6969.505353,-1,-27.729401


In [84]:
result_cosmos['Cluster'] = pred_cosmos
result_cosmos['Density'] = log_dens_cosmos
result_cosmos.to_csv('../Data/result_cosmos.csv', index= False)
result_cosmos

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z,Cluster,Density
0,COSMOS 2251,1993-036A,2024-03-14T21:31:51.966336,4103.678672,-1548.572660,-5659.902965,0,-27.635160
1,COSMOS 2251 DEB,1993-036E,2024-03-14T21:50:15.684000,-4441.067495,-2112.225840,-5206.098602,12,-27.737928
2,COSMOS 2251 DEB,1993-036F,2024-03-14T18:05:33.239616,-2413.130916,6676.674039,-461.982890,35,-28.364971
3,COSMOS 2251 DEB,1993-036G,2024-03-14T08:58:39.399168,4522.823136,-2446.781412,-4832.566767,-1,-27.885102
4,COSMOS 2251 DEB,1993-036H,2024-03-14T21:23:40.216416,-6323.053917,2989.121296,-1455.883446,1,-28.052776
...,...,...,...,...,...,...,...,...
769,COSMOS 2251 DEB,1993-036BXS,2024-03-04T05:57:55.018944,-7088.061557,-776.397735,2272.097567,-1,-28.139280
770,COSMOS 2251 DEB,1993-036BXY,2024-03-14T01:11:36.902688,-2271.767796,2840.689074,-6129.074753,4,-27.505931
771,COSMOS 2251 DEB,1993-036BYA,2024-03-14T03:53:33.007776,-4783.600914,-1240.333421,4986.096353,9,-27.826264
772,COSMOS 2251 DEB,1993-036BYD,2024-03-09T04:50:01.458528,-6378.842656,-2048.624017,2471.789528,9,-27.649742


In [85]:
result_satellite.to_csv('../Data/result_satellite.csv', index= False)
result_satellite

,OBJECT_NAME,OBJECT_ID,EPOCH,x,y,z
0,HBTSS-SV2,2024-028A,2024-03-14T20:49:05.202912,-5088.166624,5317.559842,442.135827
1,RAPTOR4,2024-028B,2024-03-14T20:47:39.831936,-5589.880528,4677.845507,1110.440119
2,RAPTOR1,2024-028C,2024-03-14T20:46:13.850112,-5996.604224,3897.539380,1791.348111
3,RAPTOR3,2024-028D,2024-03-14T22:29:38.631552,-6280.123961,2964.664983,2475.228670
4,RAPTOR2,2024-028E,2024-03-14T20:43:13.249056,-6398.693543,1997.014196,3069.606364
...,...,...,...,...,...,...
198,STARLINK-31289,2024-044X,2024-03-15T04:00:00.999936,630.943938,5502.710098,-2420.984307
199,STARLINK-31283,2024-044Y,2024-03-15T04:00:00.999936,-4633.541983,849.453021,3728.079632
200,EXO-0,2023-174DK,2024-03-14T21:03:32.576832,-4847.360303,1284.932457,4726.141361
201,2024-047A,2024-047A,2024-03-15T04:34:59.235168,2835.973022,4958.122297,3953.521985


In [86]:
result_iridium.groupby('Cluster')['Density'].mean().abs()

Cluster
-1    28.609788
 0    27.402246
 1    27.657519
 2    28.176843
 3    28.267144
 4    28.477407
 5    28.367082
 6    28.211496
Name: Density, dtype: float64

In [87]:
result_cosmos.groupby('Cluster')['Density'].mean().abs()

Cluster
-1     28.362034
 0     27.626710
 1     28.063478
 2     27.670354
 3     27.385753
 4     27.408881
 5     28.056649
 6     27.630853
 7     27.882470
 8     28.237021
 9     27.762040
 10    27.720087
 11    28.230375
 12    27.918622
 13    27.917092
 14    28.148079
 15    28.384014
 16    28.119973
 17    28.383196
 18    28.134834
 19    28.131879
 20    27.765647
 21    27.961184
 22    28.140975
 23    28.177289
 24    28.163173
 25    27.765212
 26    28.104565
 27    28.149329
 28    28.140960
 29    27.845557
 30    28.336163
 31    28.319067
 32    27.849030
 33    28.213362
 34    27.804918
 35    28.305758
 36    28.573299
Name: Density, dtype: float64

In [88]:
distances_s_i = []
for i, garbage in enumerate(data_garbage_iridium):
    for j, satellite in enumerate(data_satellite):
        distance = np.sqrt((garbage[0] - satellite[0])**2 + (garbage[1] - satellite[1])**2 + (garbage[2] - satellite[2])**2)
        distances_s_i.append((i, j, distance))

distances = pd.DataFrame(distances_s_i, columns=['garbage_index', 'satellite_index', 'distance'])
print(distances)

distances.to_csv('../Data/distances_s_i.csv', index=False)

       garbage_index  satellite_index     distance
0                  0                0  9703.685815
1                  0                1  9237.792694
2                  0                2  8744.282159
3                  0                3  8227.333755
4                  0                4  7759.605067
...              ...              ...          ...
34505            169              198  9202.944435
34506            169              199  2934.359780
34507            169              200  2265.886639
34508            169              201  6790.401942
34509            169              202  5733.017064

[34510 rows x 3 columns]


In [89]:
distances_s_c = []
for i, garbage in enumerate(data_garbage_cosmos):
    for j, satellite in enumerate(data_satellite):
        distance = np.sqrt((garbage[0] - satellite[0])**2 + (garbage[1] - satellite[1])**2 + (garbage[2] - satellite[2])**2)
        distances_s_c.append((i, j, distance))

distances = pd.DataFrame(distances_s_c, columns=['garbage_index', 'satellite_index', 'distance'])
print(distances)

distances.to_csv('../Data/distances_s_c.csv', index=False)

        garbage_index  satellite_index      distance
0                   0                0  12994.947976
1                   0                1  13363.043015
2                   0                2  13681.995231
3                   0                3  13941.773125
4                   0                4  14109.406132
...               ...              ...           ...
157117            773              198   8434.595380
157118            773              199  12116.813491
157119            773              200  13174.434473
157120            773              201  12770.322436
157121            773              202  13671.280162

[157122 rows x 3 columns]
